In [109]:
import torch
from pathlib import Path
from torchvision import transforms
from utils import ids_to_tokens, img_transformation
from modelArchitecture.modelCustomCNN import Encoder, Decoder
from GUIconverter.GUIconverter import GUIconverter
from IPython.display import display, HTML, Image
from vocab import Vocab
from PIL import Image

In [110]:
# Configuration parameters
model_file_path = "./ED--epoch-85--loss-0.01651.pth" 
img_crop_size = 224
seed = 42

# Load the saved model
loaded_model = torch.load(model_file_path)
vocab = loaded_model['vocab']

embed_size = 64
hidden_size = 256
num_layers = 2

encoder = Encoder(embed_size)
decoder = Decoder(embed_size, hidden_size, len(vocab), num_layers)

# Load model weights
encoder.load_state_dict(loaded_model["encoder_model_state_dict"])
decoder.load_state_dict(loaded_model["decoder_model_state_dict"])

<All keys matched successfully>

In [111]:
encoder.eval()

Encoder(
  (custom_cnn): CustomCNN(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout1): Dropout(p=0.15, inplace=False)
    (dropout2): Dropout(p=0.25, inplace=False)
    (dropout3): Dropout(p=0.35, inplace=False)
    (activation): LeakyReLU(negative_slope=0.01)
    (fc1): Linear(in_features=50176, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
  )
  (BatchNorm): BatchNorm1d(64, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
)

In [112]:
decoder.eval()

Decoder(
  (embed): Embedding(17, 64)
  (lstm): LSTM(64, 256, num_layers=2, batch_first=True)
  (linear): Linear(in_features=256, out_features=17, bias=True)
)

In [113]:
# Load the image
image_path = './viewer2.png'  # Change to your image's path
image = Image.open(image_path).convert('RGB')
transform = img_transformation(img_crop_size)
transformed_image = transform(image)

In [114]:
# Model prediction
features = encoder(transformed_image.unsqueeze(0))  # Unsqueeze to add batch dimension
predicted_ids = decoder.sample(features).cpu().data.numpy()
prediction = ids_to_tokens(vocab, predicted_ids)  # Assuming this function converts ids to tokens

# Convert to HTML
transpiler = GUIconverter(style='style3')
predicted_html_string = transpiler.transpile(prediction, insert_random_text=True)

In [115]:
def display_html_string(html_string):
    page = HTML(html_string)
    display(page)

In [116]:
print(predicted_html_string)

<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" integrity="sha384-QWTKZyjpPEjISv5WaRU9OFeRpok6YctnYmDr5pNlyT2bRjXh0JMhjY6hW+ALEwIH" crossorigin="anonymous">
<style>
.body { background-color: #0597F2; }
.header { margin: 20px 0; }
nav ul.nav-pills li { background-color: #F2BC57;  font-size: 20px; border-radius: 4px; margin: 10px 40px 10px -10px; padding: 10px; text-decoration: none; }
.col-lg-3 { width: 24%; margin-right: 1.333333%; }
.col-lg-6 { width: 49%; margin-right: 2%; }
.col-lg-12, .col-lg-3, .col-lg-6 { margin-bottom: 20px; border-radius: 6px; background-color: #F2D8EB; padding: 20px; }
.row .col-lg-3:last-child, .row .col-lg-6:last-child { margin-right: 0; }
footer { color:black; padding: 20px 0; text-align: center; border-top: 2px solid #D9933D; }
.btn-body { background-color: #0583F2; color: 

In [117]:
display_html_string(predicted_html_string)